In [1]:
import pandas as pd
import numpy as np

from pycta.portfolio import Portfolio
from pycta.beakerx import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
# Load prices
prices = pd.read_csv("data/prices.csv", index_col=0, parse_dates=True).ffill().truncate(before="1970-01-01")

In [3]:
# take two moving averages and apply sign-functiond
def f(price, fast=32, slow=96):
    s = price.ewm(com=slow, min_periods=300).mean()
    f = price.ewm(com=fast, min_periods=300).mean()
    return np.sign(f-s)

In [4]:
form = EasyForm("Configuration")
form.addTextField("fast")
form.addTextField("slow")
form.addButton("Go!", tag="run")
form 

In [5]:
portfolio = Portfolio(prices=prices, position=prices.apply(f, fast=int(form["fast"] or 32), slow=int(form["slow"] or 96)))
analysis(portfolio.nav())

* This is a **univariate** trading system, we map the (real) price of an asset to its (cash)position.
* Some hedge funds would argue that they are running 49 trading systems.
* Only 2 **free parameters** used here.
* Only 3 lines of code.

Results do not look terrible but...
* No concept of risk integrated. 
* The size of each bet is constant regardless of the underlying asset.
* The system lost its mojo in 2009 and has never really recovered.
* The sign function is very expensive to trade as position changes are too extreme.

Such fundamental flaws are not addressed by **parameter-hacking** or **pimp-my-trading-system** steps (remove the worst performing assets, insane quantity of stop-loss limits, ...)

In [36]:
%%time
import pandas as pd
prices = pd.read_csv("data/prices.csv", index_col=0, parse_dates=True).ffill().truncate(before="2000-01-01")
position = pd.DataFrame(index=prices.index, columns=prices.keys())#.values
for n,t in enumerate(prices.index):
    position.values[n] = prices.values[n]

CPU times: user 201 ms, sys: 12 ms, total: 213 ms
Wall time: 211 ms


In [33]:
%%time
import pandas as pd
import numpy as np
prices = pd.read_csv("data/prices.csv", index_col=0, parse_dates=True).ffill().truncate(before="2000-01-01")
index = prices.index
prices = prices.values
position = np.zeros_like(prices)
for n,t in enumerate(index):
    position[n] = prices[n]

CPU times: user 76.5 ms, sys: 12 ms, total: 88.5 ms
Wall time: 87.1 ms


In [ ]:
%%time
import pandas as pd
prices = pd.read_csv("data/prices.csv", index_col=0, parse_dates=True).ffill().truncate(before="2000-01-01")
position = pd.DataFrame(index=prices.index, columns=prices.keys())#.values
for n,t in enumerate(prices.index):
    position.loc[n] = prices.values[n]

In [ ]:
%%time
import pandas as pd
prices = pd.read_csv("data/prices.csv", index_col=0, parse_dates=True).ffill().truncate(before="2000-01-01")
position = pd.DataFrame(index=prices.index, columns=prices.keys())#.values
for n,t in enumerate(prices.index):
    position.values[n] = prices.il[n]

CPU times: user 32.5 s, sys: 15.7 ms, total: 32.5 s
Wall time: 32.5 s


In [3]:
%%time
import pandas as pd
prices = pd.read_csv("data/prices.csv", index_col=0, parse_dates=True).ffill().truncate(before="2000-01-01")
position = pd.DataFrame(index=prices.index, columns=prices.keys())#.values
for n,t in enumerate(prices.index):
    position.values[n] = prices.il[n]

CPU times: user 209 ms, sys: 8.01 ms, total: 217 ms
Wall time: 216 ms
